In [1]:
# !pip install tensorflow

In [2]:
import pandas as pd
import numpy as np
# import sklearn as sk
from matplotlib import pyplot as plt
import seaborn as sn
import seaborn as sns
from matplotlib import pyplot
# from keras.models import Sequential
# from keras.layers import Dense
# from sklearn.linear_model import LinearRegression
# from sklearn.linear_model import LogisticRegression
# from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, train_test_split, cross_val_score
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.preprocessing import LabelEncoder
# from sklearn.metrics import r2_score, explained_variance_score, confusion_matrix, accuracy_score, classification_report, log_loss
from math import sqrt

# Creating the modeling dataset
# from sklearn.datasets import make_classification
# Data processing

# Model and performance
#from sklearn.model_selection import train_test_split
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.metrics import classification_report

# Oversampling and under sampling
# from imblearn.over_sampling import RandomOverSampler, SMOTE
# from imblearn.under_sampling import RandomUnderSampler, NearMiss
# from collections import Counter
%matplotlib inline


In [3]:

# Load and preprocess the data
# data = pd.read_csv("startup_funding.csv")
# X = data.iloc[:, 0:10].values
# y = data.iloc[:, 10:].values

# Define the neural network architecture
# model = Sequential()
# model.add(Dense(64, input_dim=10, activation='relu'))
# model.add(Dense(32, activation='relu'))
# model.add(Dense(16, activation='relu'))
# model.add(Dense(8, activation='softmax'))

# Compile the model
# model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
# model.fit(X, y, epochs=50, batch_size=32, validation_split=0.2)

# Generate predictions for a new startup
# new_startup = np.array([[1000000, 0, 1, 0, 0, 0, 1, 0, 0, 0]])
# predictions = model.predict(new_startup)

# Select the output nodes with the highest activation levels
# investors = np.argsort(predictions)[0][-15:]

In [4]:
# Load and preprocess the data
# data = pd.read_csv("startup_funding.csv")
# X = data.iloc[:, 0:10].values
# investor_names = data.columns[10:].tolist()
# y = data.iloc[:, 10:].values

# Define the neural network architecture
# model = Sequential()
# model.add(Dense(64, input_dim=10, activation='relu'))
# model.add(Dense(32, activation='relu'))
# model.add(Dense(16, activation='relu'))
# model.add(Dense(len(investor_names), activation='softmax'))

# Compile the model
# model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
# model.fit(X, y, epochs=50, batch_size=32, validation_split=0.2)

# Generate predictions for a new startup
# new_startup = np.array([[1000000, 0, 1, 0, 0, 0, 1, 0, 0, 0]])
# predictions = model.predict(new_startup)

# Map the numeric output to investor names
# investor_probs = {name: prob for name, prob in zip(investor_names, predictions[0])}
# top_investors = sorted(investor_probs, key=investor_probs.get, reverse=True)[:15]


In [5]:
# df_deals = pd.read_excel ('../data/Deals (investment).xlsx')
# df_deals.head()

train = pd.read_csv ('train.csv')
train.shape

(726, 14)

In [6]:
test = pd.read_csv ('train.csv')
test.head()

,6_investors,"Deals Information, Level of Completeness",Disclosed,Founded,Total Disclosed Funding,DA Classification_African Company (Yes = 0; No = 1),Year,Month,Quarter,Half,Number of Investors,main_sector_encoded,funding_round_encoded,cat_Country
0,First Light Ventures,0.9,1.0,2011.0,6504900.0,0.0,2010.0,8.0,3.0,2.0,1.0,1,1,10
1,Tim Draper,0.9,1.0,2009.0,32700000.0,0.0,2010.0,9.0,3.0,2.0,1.0,1,1,19
2,Undisclosed,0.9,1.0,2009.0,7850000.0,0.0,2010.0,12.0,4.0,2.0,1.0,2,2,5
3,Omidyar Network,0.9,1.0,2008.0,55800000.0,0.0,2011.0,5.0,2.0,1.0,1.0,3,2,23
4,TBL Mirror Fund,0.9,1.0,2004.0,54500000.0,0.0,2011.0,10.0,4.0,2.0,1.0,1,3,10


In [7]:
# Seperating Predictors and Outcome values from train and test sets
X_train = pd.DataFrame(train.drop(['6_investors'],axis=1))
Y_train_label = train['6_investors'].values.astype(object)
X_test = pd.DataFrame(test.drop(['6_investors'],axis=1))
Y_test_label = test['6_investors'].values.astype(object)


In [8]:
# Transforming non numerical labels into numerical labels
from sklearn import preprocessing
encoder = preprocessing.LabelEncoder()

# encoding train labels 
encoder.fit(Y_train_label)
Y_train = encoder.transform(Y_train_label)

# encoding test labels 
encoder.fit(Y_test_label)
Y_test = encoder.transform(Y_test_label)


In [9]:
#Total Number of Continous and Categorical features in the training set
num_cols = X_train._get_numeric_data().columns
print("Number of numeric features:",num_cols.size)
#list(set(X_train.columns) - set(num_cols))
num_cols

Number of numeric features: 13


Index(['Deals Information, Level of Completeness', 'Disclosed ', 'Founded',
       'Total  Disclosed Funding',
       'DA Classification_African Company (Yes = 0; No = 1)', 'Year', 'Month',
       'Quarter', 'Half', 'Number of Investors', 'main_sector_encoded',
       'funding_round_encoded', 'cat_Country'],
      dtype='object')

In [10]:
names_of_predictors = list(X_train.columns.values)

# Scaling the Train and Test feature set 
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

#### Hyperparameter tuning using grid search and cross validation

In [11]:
#Libraries to Build Ensemble Model : Random Forest Classifier 
# Create the parameter grid based on the results of random search 
params_grid = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
                     'C': [1, 10, 100, 1000]},
                    {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]

In [12]:
# kernel = ['linear','rbf','poly','sigmoid']
# for i in kernel:
#     svm_model = SVC(kernel=i, C=1.0)
#     svm_model.fit(X_train_scaled, Y_train)
    

In [13]:
# print('for kennel: ', i)
# print('accuracy is : ', svm_model.score(X_train_scaled , Y_train))

#### Training SVM model using radial kernel

In [15]:
# Performing CV to tune parameters for best SVM fit 
from sklearn.svm import SVC
svm_model = GridSearchCV(SVC(), params_grid, cv=5)
svm_model.fit(X_train_scaled, Y_train)

/home/nicholas/.local/lib/python3.10/site-packages/sklearn/model_selection/_split.py:684: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


GridSearchCV(cv=5, estimator=SVC(),
             param_grid=[{'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001],
                          'kernel': ['rbf']},
                         {'C': [1, 10, 100, 1000], 'kernel': ['linear']}])

#### Confusion Matrix and Accuracy Score


In [16]:
# View the accuracy score
print('Best score for training data:', svm_model.best_score_,"\n") 

# View the best parameters for the model found using grid search
print('Best C:',svm_model.best_estimator_.C,"\n") 
print('Best Kernel:',svm_model.best_estimator_.kernel,"\n")
print('Best Gamma:',svm_model.best_estimator_.gamma,"\n")

final_model = svm_model.best_estimator_
Y_pred = final_model.predict(X_test_scaled)
Y_pred_label = list(encoder.inverse_transform(Y_pred))

Best score for training data: 0.1625413320736892 

Best C: 1 

Best Kernel: rbf 

Best Gamma: 0.001 



In [17]:
# Making the Confusion Matrix
#print(pd.crosstab(Y_test_label, Y_pred_label, rownames=['Actual investors'], colnames=['Predicted investors']))
# print(confusion_matrix(Y_test_label,Y_pred_label))
print("\n")
print(classification_report(Y_test_label,Y_pred_label))

print("Training set score for SVM: %f" % final_model.score(X_train_scaled , Y_train))
print("Testing  set score for SVM: %f" % final_model.score(X_test_scaled  , Y_test ))

svm_model.score

/home/nicholas/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/nicholas/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/nicholas/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                                                                                                                                                                                                         precision    recall  f1-score   support

                                                                                                                                                                                                BoostVC       0.00      0.00      0.00         1
                                                                                                                                                                                        Flat6Labs Cairo       0.00      0.00      0.00         1
                                                                                                                                                                                          1776 Ventures       0.00      0.00      0.00         1
                                   

<bound method BaseSearchCV.score of GridSearchCV(cv=5, estimator=SVC(),
             param_grid=[{'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001],
                          'kernel': ['rbf']},
                         {'C': [1, 10, 100, 1000], 'kernel': ['linear']}])>

In [18]:
# cols = df_deals.columns

In [19]:
# for cat_col in cols:
#     print(f"{cat_col}: {df_deals[cat_col].nunique()} uniqueness variable")

In [20]:
# df_deals.shape

In [ ]:
# df_deals['Investor 2'].unique()

In [ ]:
# df_deals.columns

In [ ]:
# df_deals = df_deals.drop(['Investor 1','Cummulative Deals (Disclosed)','Cummulative Deals (Undisclosed)','Cummulative Amount','7_source','4_Stake','Investor 2', 'Investor 3',
#        'Investor 4', 'Investor 5', 'Investor 6', 'Investor 7', 'Investor 8',
#        'Investor 9', 'Investor 10', 'Investor 11', 'Investor 12','Investor 13','Investor 14','Investor 13.1','Investor 14.1','Investor 15.1','Investor 15','Duplication','Investor 1.1',
#        'Investor 2.1', 'Investor 3.1', 'Investor 4.1', 'Investor 5.1',
#        'Investor 6.1', 'Investor 7.1', 'Investor 8.1', 'Investor 9.1',
#        'Investor 10.1', 'Investor 11.1', 'Investor 12.1'], axis=1)

In [ ]:
# df_deals.head()

In [ ]:
# df_deals["DA Classification_African Company (Yes = 0; No = 1)"].value_counts()

In [ ]:
# df_deals = df_deals.dropna()

In [ ]:
# df_deals.head()

In [ ]:
# df_deals.shape

In [ ]:
# def get_year(dt):
#     return dt.year

# df_deals['year'] = df_deals['1_post_date'].map(get_year)
# #df_deals

In [ ]:
# df_deals

In [ ]:
# df_deals = df_deals.drop(['Year','year','Country/Town'], axis=1)

In [ ]:
# df_deals

In [ ]:
# #Dropping columns that the startup wont have prior
# df_deals = df_deals.drop(['1_post_date','2_post_title','Check vs Companies','Industry, DA'], axis=1)

In [ ]:
# df_deals

In [ ]:
# df_deals = df_deals.drop(['duplicated_conc'], axis=1)

In [ ]:
# df_deals['5_funding_round'].unique()

In [ ]:
# !pip install category_encoders

In [ ]:
# from category_encoders import OrdinalEncoder

In [ ]:
# df_deals['main_sector'].unique()

In [ ]:
# df_deals['main_sector'] = df_deals['main_sector'].apply(lambda x: x.lower())
# df_deals['main_sector'].unique()

In [ ]:
# def main_sector(sector):
#     if 'recruitment' in sector.lower():
#         return 'recruitment, human resource'
#     elif 'media/ entertainment' in sector.lower():
#         return 'media and entertainment'
#     elif 'media & entertainment' in sector.lower():
#         return 'media and entertainment'
#     elif 'solar power' in sector.lower():
#         return 'solar energy, energy and resources'
#     elif 'real estate' in sector.lower():
#         return 'real estate and construction'
#     elif 'resource & energy' in sector.lower():
#         return 'solar energy, energy and resources'
#     elif 'telecommunications' in sector.lower():
#         return 'utilities and telecommunication services'
#     elif 'renewable energy' in sector.lower():
#         return 'renewables & environment'
#     elif 'information technology' in sector.lower():
#         return 'other technologies & information technology'
#     else:
#         return sector

In [ ]:
# #applying our function to eliminate duplicates
# df_deals['main_sector'] = df_deals['main_sector'].apply(main_sector)

In [ ]:
# df_deals['main_sector'].unique()

In [ ]:
# mapping_ms = [{'col': 'main_sector', 'mapping': {'financial services': 1, 'recruitment, human resource': 2, 'education': 3,
#        'media and entertainment': 4, 'e-commerce & retail': 5,
#        'other technologies & information technology': 6,
#        'real estate and construction': 7,
#        'commercial & professional services': 8, 'software': 9,
#        'transport & logistics': 10, 'automotive': 11, 'healthcare & pharma': 12,
#        'utilities and telecommunication services': 13, 'agriculture': 14,
#        'travel & leisure': 15, 'social network': 16, 'security': 17,
#        'renewables & environment': 18, 'emerging technologies': 19, 'saas': 20,
#        'marketing': 21, 'solar energy, energy and resources': 22, 'robotics': 23,
#        'travel and leisure': 24}}]
# OrdinalEncoder(cols=['main_sector'], mapping=mapping_ms).fit(df_deals).transform(df_deals)

# mapping_ms = {'financial services': 1, 'recruitment, human resource': 2, 'education': 3,
#        'media and entertainment': 4, 'e-commerce & retail': 5,
#        'other technologies & information technology': 6,
#        'real estate and construction': 7,
#        'commercial & professional services': 8, 'software': 9,
#        'transport & logistics': 10, 'automotive': 11, 'healthcare & pharma': 12,
#        'utilities and telecommunication services': 13, 'agriculture': 14,
#        'travel & leisure': 15, 'social network': 16, 'security': 17,
#        'renewables & environment': 18, 'emerging technologies': 19, 'saas': 20,
#        'marketing': 21, 'solar energy, energy and resources': 22, 'robotics': 23,
#        'travel and leisure': 24}

# df_deals['main_sector_encoded'] = df_deals['main_sector'].apply(lambda x: mapping_ms[x])

In [ ]:
# df_deals.head()

In [ ]:
# mapping_sr = [{'col': '5_funding_round', 'mapping': {'Seed': 1, 'Grant': 2, 'Series A': 3, 'Angel': 4, 'Debt Financing': 5, 'Series B': 6,
#        'Series D': 7, 'Series C': 8, 'Private Equity': 9, 'Pre-Series B': 10,
#        'Convertible Note': 11, 'Pre-Seed': 12, 'Venture Round': 13, 'Series E': 14,
#        'Undisclosed': 15, 'Crowdfunding': 16, 'Corporate Venture': 17,
#        'Corporate Round': 18, 'Series A II': 19, 'CrowdFunding': 20}}]
# OrdinalEncoder(cols=['5_funding_round'], mapping=mapping_sr).fit(df_deals).transform(df_deals)


In [ ]:
# mapping_sr = {'Seed': 1, 'Grant': 2, 'Series A': 3, 'Angel': 4, 'Debt Financing': 5, 'Series B': 6,
#        'Series D': 7, 'Series C': 8, 'Private Equity': 9, 'Pre-Series B': 10,
#        'Convertible Note': 11, 'Pre-Seed': 12, 'Venture Round': 13, 'Series E': 14,
#        'Undisclosed': 15, 'Crowdfunding': 16, 'Corporate Venture': 17,
#        'Corporate Round': 18, 'Series A II': 19, 'CrowdFunding': 20}

# df_deals['funding_round_encoded'] = df_deals['5_funding_round'].apply(lambda x: mapping_sr[x])

In [ ]:
# df_deals.head()

In [ ]:
# df_deals['Category'].unique()

In [ ]:
# df_deals.head()

In [ ]:
# Load and preprocess the data
data = df_deals
X = data.iloc[:, 0:2].values
# Y_train_label = train.Activity.values.astype(object)
# investor_names = data.columns[10:].tolist()
y = data.iloc[:, 2:].values

In [ ]:
data = data.rename(columns={' Disclosed': "Disclosed"})

In [ ]:
mode_value = data['6_investors'].mode()[0]


In [ ]:
# data['6_investors'].fillna(mode_value, inplace=True)
# data['6_investors'] = data['6_investors'].replace('Undisclosed', mode_value, inplace=True)

data


In [ ]:
# the funding total column is read as a object so clearning it up so that we can use it as a numerical column
data['5_funding_round']=data['5_funding_round'].str.replace(',','') # removing commas from funding_total_usd column
data['5_funding_round']=data['5_funding_round'].str.replace(' ','')#removing extra space from funding_total_usd column
# df['funding_total_usd']=df['funding_total_usd'].str.replace('-','0')
#removing - from funding_total_usd column and replacing with 0

In [ ]:
data['5_funding_round'].unique()

In [ ]:
data = data.drop(['Category','funding round, DA'], axis=1)

In [ ]:
# data

In [ ]:
data['5_funding_round'].unique()

In [ ]:
# data['cat_funding_round'] = data['5_funding_round'].replace(['Seed', 'Grant', 'SeriesA', 'Angel', 'DebtFinancing', 'SeriesB',
#        'SeriesD', 'SeriesC', 'PrivateEquity', 'Pre-SeriesB',
#        'ConvertibleNote', 'Pre-Seed', 'VentureRound', 'SeriesE',
#        'Undisclosed', 'Crowdfunding', 'CorporateVenture',
#        'CorporateRound', 'SeriesAII', 'CrowdFunding'], [0, 1, 2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19])

In [ ]:
data

In [ ]:
# creating instance of labelencoder
labelencoder = LabelEncoder()
#using label encoder for these two columns as there is a lot of variables
data['cat_Country'] = labelencoder.fit_transform(data['Country(HQ)']) # using label encoder on continent
# data['cat_6_investors'] = labelencoder.fit_transform(data['6_investors']) # using label encoder on industry group
# data['cat_main_sector'] = labelencoder.fit_transform(data['main_sector'])

### prepare data for modelling

In [ ]:
data.columns

In [ ]:
data1 = data[['Deals Information, Level of Completeness', 'Disclosed ',
       'Founded', 'Total  Disclosed Funding',
       'DA Classification_African Company (Yes = 0; No = 1)', 'Month',
       'Quarter', 'Half', 'Number of Investors', 'main_sector_encoded',
       'funding_round_encoded', 'cat_Country']]

In [ ]:
def get_data_types(df):
    """
    Get the data types of all columns in a dataframe.

    Args:
        df (pandas.DataFrame): The dataframe to retrieve data types from.

    Returns:
        dict: A dictionary mapping column names to their respective data types.
    """
    data_types = df.dtypes.to_dict()
    return data_types

In [ ]:
data_types = get_data_types(data1)

# Print the data types
print(data_types)

#### Statitics and correlation

In [ ]:
#creating correlation matrix
colormap = plt.cm.viridis
plt.figure(figsize = (35, 35))
plt.title('Pearson Correlation of features', y = 1.05, size = 15)
matrix = np.triu(data1.corr())
sns.heatmap(data1.astype(float).corr(), linewidth = 0.1, vmax = 1.0, square =True, cmap=colormap, linecolor = 'white', annot=True, mask = matrix)


In [ ]:
data = data.drop(['3_amount','5_funding_round','Country(HQ)','main_sector'], axis=1)

In [ ]:
data

In [ ]:
Y = data['6_investors'] #setting Y variable
X = data.drop('6_investors', axis = 1) #dropping status and setting features
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 42)#test and train dataset
Y_train_label = Y_train.values.astype(object)
Y_test_label = Y_test.values.astype(object)
# Y_train.shape
# X_test.shape
# Y_test.shape
X_test.head()
Y_train_label

In [ ]:
#checking size of each dataset
# print('Shape of X_train=>',X_train.shape)
# print('Shape of X_test=>',X_test.shape)
# print('Shape of Y_train=>',Y_train.shape)
# print('Shape of Y_test=>',Y_test.shape)

In [ ]:
# Transforming non numerical labels into numerical labels
# from sklearn import preprocessing
# encoder = preprocessing.LabelEncoder()

# # encoding train labels 
# encoder.fit(Y_train_label)
# Y_train = encoder.transform(Y_train_label)

# # encoding test labels 
# encoder.fit(Y_test_label)
# Y_test = encoder.transform(Y_test_label)

# Y_train.shape

In [ ]:
# Scaling the Train and Test feature set 
# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(X_train)
# X_test_scaled = scaler.transform(X_test)
# # X_train.shape
# # X_train.shape
# X_test_scaled.shape

In [ ]:
#Libraries to Build Ensemble Model : Random Forest Classifier 
# # Create the parameter grid based on the results of random search 
# params_grid = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
#                      'C': [1, 10, 100, 1000]},
#                     {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]

In [ ]:
# from sklearn.svm import SVC

In [ ]:
# # Performing CV to tune parameters for best SVM fit 
# svm_model = GridSearchCV(SVC(), params_grid, cv=5)
# svm_model.fit(X_train_scaled, Y_train)
# # Y_train.shape

#### Confusion Matrix and Accuracy Score

In [ ]:
# # View the accuracy score
# print('Best score for training data:', svm_model.best_score_,"\n") 

# # View the best parameters for the model found using grid search
# print('Best C:',svm_model.best_estimator_.C,"\n") 
# print('Best Kernel:',svm_model.best_estimator_.kernel,"\n")
# print('Best Gamma:',svm_model.best_estimator_.gamma,"\n")

# final_model = svm_model.best_estimator_
# Y_pred = final_model.predict(X_test_scaled)
# # Y_pred
# Y_pred_label = list(encoder.inverse_transform(Y_pred))

In [ ]:
# Making the Confusion Matrix
#print(pd.crosstab(Y_test_label, Y_pred_label, rownames=['Actual Activity'], colnames=['Predicted Activity']))
# print(confusion_matrix(Y_test_label,Y_pred_label))
# print("\n")
# print(classification_report(Y_test_label,Y_pred_label))

# print("Training set score for SVM: %f" % final_model.score(X_train_scaled , Y_train))
# print("Testing  set score for SVM: %f" % final_model.score(X_test_scaled  , Y_test ))

# svm_model.score

In [ ]:
# names_of_predictors = list(X_train.columns.values)

In [ ]:
# names_of_predictors

In [ ]:
# Scaling the Train and Test feature set 
# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(X_train)
# X_test_scaled = scaler.transform(X_test)

##### Hyperparameter tuning using grid search and cross validation

In [ ]:
#Libraries to Build Ensemble Model : Random Forest Classifier 
# Create the parameter grid based on the results of random search 
# params_grid = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
#                      'C': [1, 10, 100, 1000]},
#                     {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]

#### Training SVM model using radial kernel

In [ ]:
# from sklearn.utils import shuffle
# from sklearn.svm import SVC
# from sklearn.metrics import confusion_matrix,classification_report
# from sklearn.model_selection import cross_val_score, GridSearchCV
# import numpy as np

In [ ]:
# Y_train

In [ ]:
# Y_train.head()

# x = np.asfarray(Y_train)
# X_train

In [ ]:
# Performing CV to tune parameters for best SVM fit 
# svm_model = GridSearchCV(SVC(), params_grid, cv=5)
# svm_model.fit(X_train_scaled, Y_train)